# Homework 3

In [2]:
import pandas as pd
import numpy as np
import wget

In [3]:
data = wget.download('https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv')

100% [..........................................................................] 1475504 / 1475504

In [76]:
df = pd.read_csv(data)

In [77]:
df.columns

Index(['Make', 'Model', 'Year', 'Engine Fuel Type', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Driven_Wheels',
       'Number of Doors', 'Market Category', 'Vehicle Size', 'Vehicle Style',
       'highway MPG', 'city mpg', 'Popularity', 'MSRP'],
      dtype='object')

## Data preparation

In [78]:
df = df[['Make', 'Model', 'Year', 'Engine HP', 'Engine Cylinders', 'Transmission Type', 'Vehicle Style', 'highway MPG',
        'city mpg', 'MSRP']]

In [79]:
df.columns = df.columns.str.replace(' ', '_').str.lower()

In [80]:
# Fill in the missing values of the selected features with 0.
df = df.fillna(0)

In [81]:
# Rename MSRP variable to price.
df.rename(columns = {'msrp':'price'}, inplace = True)

## Question 1

What is the most frequent observation (mode) for the column transmission_type?

In [82]:
df['transmission_type'].mode()

0    AUTOMATIC
dtype: object

Answer: AUTOMATIC

## Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.

What are the two features that have the biggest correlation in this dataset?

In [83]:
df.corr()

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.000000,0.338714,-0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306,0.526274
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000,-0.157676
price,0.227590,0.650095,0.526274,-0.160043,-0.157676,1.000000


Answer: highway_mpg and city_mpg

## Make price binary

Now we need to turn the price variable from numeric into a binary format.  
Let's create a variable above_average which is 1 if the price is above its mean value and 0 otherwise.

In [84]:
df['above_average'] = (df['price'] > df['price'].mean()).astype(int)

## Split the data

Split your data in train/val/test sets with 60%/20%/20% distribution.  
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.  
Make sure that the target value (above_average) is not in your dataframe.

In [85]:
from sklearn.model_selection import train_test_split

In [86]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [87]:
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

In [88]:
len(df_train), len(df_val), len(df_test)

(7148, 2383, 2383)

In [89]:
df_train = df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [90]:
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

In [93]:
del df_train['above_average']
del df_val['above_average']
del df_test['above_average']

## Question 3

Calculate the mutual information score between above_average and other categorical variables in our dataset. Use the training set only.  
Round the scores to 2 decimals using round(score, 2).  
Which of these variables has the lowest mutual information score?

In [102]:
df_train_full = df_train_full.reset_index(drop = True)

In [103]:
df_train_full.above_average.value_counts(normalize = True)

0    0.723219
1    0.276781
Name: above_average, dtype: float64

In [104]:
df_train_full.dtypes

make                  object
model                 object
year                   int64
engine_hp            float64
engine_cylinders     float64
transmission_type     object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
price                  int64
above_average          int32
dtype: object

In [107]:
df_train_full.nunique()

make                   48
model                 902
year                   28
engine_hp             349
engine_cylinders        9
transmission_type       5
vehicle_style          16
highway_mpg            58
city_mpg               66
price                5266
above_average           2
dtype: int64

In [105]:
from sklearn.metrics import mutual_info_score

In [113]:
print('make:', round(mutual_info_score(df_train_full.above_average, df_train_full.make), 2))
print('model:', round(mutual_info_score(df_train_full.above_average, df_train_full.model), 2))
print('transmission_type:', round(mutual_info_score(df_train_full.above_average, df_train_full.transmission_type), 2))
print('vehicle_style:', round(mutual_info_score(df_train_full.above_average, df_train_full.vehicle_style), 2))

make: 0.24
model: 0.46
transmission_type: 0.02
vehicle_style: 0.08


Answer: transmission_type